In [1]:
import os
from PIL import Image
import numpy as np
from tensorflow.keras import backend 
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2025-05-10 20:49:18.289707: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Loading data

In [2]:
spectrogram = '/home/madan005/dev/deeplearning/MFCC'


train_spectrogram      = os.path.join(spectrogram, 'train')
validation_spectrogram = os.path.join(spectrogram, 'dev')
test_spectrogram       = os.path.join(spectrogram, 'eval')


def load_images_from_folder(folder, label, target_size=(200, 200)):
    images = []
    labels = []
    for filename in os.listdir(folder):
        filepath = os.path.join(folder, filename)
        image = Image.open(filepath)
        image = image.convert('RGB')
        # resizing
        image = image.resize(target_size, Image.LANCZOS)
        images.append(np.array(image))
        labels.append(label)
    return images, labels

# data lists
X_train, y_train = [], []
X_val,   y_val   = [], []
X_test,  y_test  = [], []

# training data
for class_name in ['genuine', 'spoof']:
    class_folder = os.path.join(train_spectrogram, class_name)
    label = 1 if class_name == 'genuine' else 0
    imgs, labels = load_images_from_folder(class_folder, label)
    X_train.extend(imgs)
    y_train.extend(labels)  
# validation data
for class_name in ['genuine', 'spoof']:
    class_folder = os.path.join(validation_spectrogram, class_name)
    label = 1 if class_name == 'genuine' else 0
    imgs, labels = load_images_from_folder(class_folder, label)
    X_val.extend(imgs)
    y_val.extend(labels)
# test data
for class_name in ['genuine', 'spoof']:
    class_folder = os.path.join(test_spectrogram, class_name)
    label = 1 if class_name == 'genuine' else 0
    imgs, labels = load_images_from_folder(class_folder, label)
    X_test.extend(imgs)
    y_test.extend(labels)



# Normalizing the Images (Min-Max Scaling)
X_train = np.array(X_train, dtype='float32') / 255.0
X_val   = np.array(X_val,   dtype='float32') / 255.0
X_test  = np.array(X_test,  dtype='float32') / 255.0


y_train = np.array(y_train)
y_val   = np.array(y_val)
y_test  = np.array(y_test)

# Shapes
print("Training set:", X_train.shape)
print("Validation set:", X_val.shape)
print("Test set:", X_test.shape)

Training set: (3014, 200, 200, 3)
Validation set: (1710, 200, 200, 3)
Test set: (13306, 200, 200, 3)


class distributions

In [3]:
# class counts 
for split_name, labels in [
    ('Training', y_train),
    ('Validation', y_val),
    ('Test', y_test)
]:
    unique, counts = np.unique(labels, return_counts=True)
    print(f"\n{split_name}:")
    for cls, cnt in zip(unique, counts):
        name = 'genuine' if cls == 1 else 'spoof'
        print(f"  {name}: {cnt}")



Training:
  spoof: 1507
  genuine: 1507

Validation:
  spoof: 950
  genuine: 760

Test:
  spoof: 12008
  genuine: 1298


In [4]:

IMG_SIZE    = (200, 200)
BATCH_SIZE  = 32
LR          = 1e-4
L2_REG      = 1e-4


# augmentation
train_aug = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
)

# MobileNetV2 backbone
base_model = MobileNetV2(include_top=False, weights='imagenet', input_shape=(*IMG_SIZE, 3), pooling='avg')

base_model.trainable = False   # freeze all layers

#classification head
inputs = layers.Input(shape=(*IMG_SIZE, 3))
x = base_model(inputs, training=False)
# x = layers.Dropout(0.2)(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)



/tmp/ipykernel_4083379/4144618430.py:17: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(include_top=False, weights='imagenet', input_shape=(*IMG_SIZE, 3), pooling='avg')


In [5]:
model = models.Model(inputs, outputs)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 200, 200, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 1280)           │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │        81,984 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,340,033 (8.93 MB)

 Trainable params: 82,049 (320.50 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [6]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=5, restore_best_weights=True )]

#Train
history = model.fit(
    train_aug.flow(X_train, y_train, batch_size=BATCH_SIZE),
    validation_data=(X_val, y_val),
    epochs=50,
    callbacks=callbacks
)


/home/madan005/miniconda3/envs/my_env/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
95/95 ━━━━━━━━━━━━━━━━━━━━ 31s 297ms/step - accuracy: 0.5402 - loss: 0.7024 - val_accuracy: 0.5006 - val_loss: 0.7007
Epoch 2/50
95/95 ━━━━━━━━━━━━━━━━━━━━ 36s 376ms/step - accuracy: 0.6095 - loss: 0.6563 - val_accuracy: 0.4906 - val_loss: 0.7240
Epoch 3/50
95/95 ━━━━━━━━━━━━━━━━━━━━ 38s 344ms/step - accuracy: 0.6267 - loss: 0.6499 - val_accuracy: 0.4953 - val_loss: 0.7434
Epoch 4/50
95/95 ━━━━━━━━━━━━━━━━━━━━ 33s 261ms/step - accuracy: 0.6416 - loss: 0.6373 - val_accuracy: 0.4953 - val_loss: 0.7332
Epoch 5/50
95/95 ━━━━━━━━━━━━━━━━━━━━ 20s 209ms/step - accuracy: 0.6503 - loss: 0.6300 - val_accuracy: 0.4912 - val_loss: 0.7593
Epoch 6/50
95/95 ━━━━━━━━━━━━━━━━━━━━ 20s 211ms/step - accuracy: 0.6501 - loss: 0.6181 - val_accuracy: 0.4842 - val_loss: 0.7866


In [7]:
#  unfreeze 20 MobileNetV2’s top layers
base_model.trainable = True
for layer in base_model.layers[:-20]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 200, 200, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 1280)           │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │        81,984 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,340,033 (8.93 MB)

 Trainable params: 1,134,529 (4.33 MB)

 Non-trainable params: 1,205,504 (4.60 MB)

In [8]:


history_fine = model.fit(
    train_aug.flow(X_train, y_train, batch_size=BATCH_SIZE),
    validation_data=(X_val, y_val),
    epochs=20,
    callbacks=callbacks
)

Epoch 1/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 25s 228ms/step - accuracy: 0.5356 - loss: 0.7026 - val_accuracy: 0.5211 - val_loss: 0.6939
Epoch 2/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 23s 238ms/step - accuracy: 0.6094 - loss: 0.6594 - val_accuracy: 0.5351 - val_loss: 0.6957
Epoch 3/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 20s 215ms/step - accuracy: 0.6248 - loss: 0.6394 - val_accuracy: 0.5322 - val_loss: 0.6965
Epoch 4/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 21s 220ms/step - accuracy: 0.6351 - loss: 0.6335 - val_accuracy: 0.5409 - val_loss: 0.6968
Epoch 5/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 21s 217ms/step - accuracy: 0.6970 - loss: 0.5965 - val_accuracy: 0.5404 - val_loss: 0.6975
Epoch 6/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 21s 219ms/step - accuracy: 0.6610 - loss: 0.6010 - val_accuracy: 0.5181 - val_loss: 0.7029


In [9]:

y_pred_train_raw = model.predict(X_train)
y_pred_val_raw   = model.predict(X_val)
y_pred_test_raw  = model.predict(X_test)

# 2) Convert to discrete labels
def to_labels(probs):
    # binary-probabilities => threshold at 0.5
    return (probs > 0.5).astype(int).ravel()

y_pred_train = to_labels(y_pred_train_raw)
y_pred_val   = to_labels(y_pred_val_raw)
y_pred_test  = to_labels(y_pred_test_raw)

# 3) Define class names
target_names = ['spoof', 'genuine']

# 4) Loop through each split
for split_name, y_true, y_pred in [
    ('Train',      y_train, y_pred_train),
    ('Validation', y_val,   y_pred_val),
    ('Test',       y_test,  y_pred_test)
]:
    print(f"\n=== Classification Report — {split_name} ===")
    print(classification_report(y_true, y_pred, target_names=target_names))
    


95/95 ━━━━━━━━━━━━━━━━━━━━ 9s 90ms/step
54/54 ━━━━━━━━━━━━━━━━━━━━ 6s 117ms/step
  1/416 ━━━━━━━━━━━━━━━━━━━━ 1:09 167ms/step

2025-05-10 20:55:04.038136: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 6386880000 exceeds 10% of free system memory.


416/416 ━━━━━━━━━━━━━━━━━━━━ 82s 197ms/step

=== Classification Report — Train ===
              precision    recall  f1-score   support

       spoof       0.51      1.00      0.67      1507
     genuine       0.97      0.02      0.05      1507

    accuracy                           0.51      3014
   macro avg       0.74      0.51      0.36      3014
weighted avg       0.74      0.51      0.36      3014


=== Classification Report — Validation ===
              precision    recall  f1-score   support

       spoof       0.56      0.62      0.59       950
     genuine       0.46      0.39      0.42       760

    accuracy                           0.52      1710
   macro avg       0.51      0.51      0.51      1710
weighted avg       0.51      0.52      0.52      1710


=== Classification Report — Test ===
              precision    recall  f1-score   support

       spoof       0.90      0.97      0.94     12008
     genuine       0.13      0.04      0.06      1298

    accuracy     